<a href="https://colab.research.google.com/github/tomhruby1/zdo/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
import math
import json
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from pathlib import Path
import torch
import torch.nn.functional as F
import copy

In [2]:


def intersectLines( pt1, pt2, ptA, ptB ):
    """ this returns the intersection of Line(pt1,pt2) and Line(ptA,ptB)

        returns a tuple: (xi, yi, valid, r, s), where
        (xi, yi) is the intersection
        r is the scalar multiple such that (xi,yi) = pt1 + r*(pt2-pt1)
        s is the scalar multiple such that (xi,yi) = pt1 + s*(ptB-ptA)
            valid == 0 if there are 0 or inf. intersections (invalid)
            valid == 1 if it has a unique intersection ON the segment    """

    DET_TOLERANCE = 0.00000001

    # the first line is pt1 + r*(pt2-pt1)
    # in component form:
    x1, y1 = pt1;   x2, y2 = pt2
    dx1 = x2 - x1;  dy1 = y2 - y1

    # the second line is ptA + s*(ptB-ptA)
    x, y = ptA;   xB, yB = ptB;
    dx = xB - x;  dy = yB - y;

    # we need to find the (typically unique) values of r and s
    # that will satisfy
    #
    # (x1, y1) + r(dx1, dy1) = (x, y) + s(dx, dy)
    #
    # which is the same as
    #
    #    [ dx1  -dx ][ r ] = [ x-x1 ]
    #    [ dy1  -dy ][ s ] = [ y-y1 ]
    #
    # whose solution is
    #
    #    [ r ] = _1_  [  -dy   dx ] [ x-x1 ]
    #    [ s ] = DET  [ -dy1  dx1 ] [ y-y1 ]
    #
    # where DET = (-dx1 * dy + dy1 * dx)
    #
    # if DET is too small, they're parallel
    #
    DET = (-dx1 * dy + dy1 * dx)

    if math.fabs(DET) < DET_TOLERANCE: return (0,0,0,0,0)

    # now, the determinant should be OK
    DETinv = 1.0/DET

    # find the scalar amount along the "self" segment
    r = DETinv * (-dy  * (x-x1) +  dx * (y-y1))

    # find the scalar amount along the input line
    s = DETinv * (-dy1 * (x-x1) + dx1 * (y-y1))

    # return the average of the two descriptions
    xi = (x1 + r*dx1 + x + s*dx)/2.0
    yi = (y1 + r*dy1 + y + s*dy)/2.0

    ##############
    #found is intersection (xi,yi) in inner segment
    valid = 0
    if x1 != x2:
        if x1 < x2:
            a = x1
            b = x2
        else:
            a = x2
            b = x1
        c = xi
    else:
        #predpoklad, ze pak y jsou ruzne
        if y1 < y2:
            a = y1
            b = y2
        else:
            a = y2
            b = y1
        c = yi
    if (c > a) and (c < b):
        #now second segment
        if x != xB:
            if x < xB:
                a = x
                b = xB
            else:
                a = xB
                b = x
            c = xi
        else:
            #predpoklad, ze pak y jsou ruzne
            if y < yB:
                a = y
                b = yB
            else:
                a = yB
                b = y
            c = yi
        if (c > a) and (c < b):
            valid = 1

    return ( xi, yi, valid, r, s )


In [3]:
def slope(x1, y1, x2, y2): # Line slope given two points:
    return (y2-y1)/(x2-x1)

def angle(s1, s2): 
    return math.degrees(math.atan((s2-s1)/(1+(s2*s1))))

In [4]:
def visualize(data, image_id):
    '''Visualizes image and it's Incission and Stitch annotations'''


    COLOR = {
    'Incision': 'red',
    'Stitch': 'green',
    'incision': 'red',
    'stitch': 'green'
    }
    image_path = data[image_id]['file']
    image = mpimg.imread(image_path)
    
    fig, ax = plt.subplots()
    ax.imshow(image)

    for idx, label in enumerate(data[image_id]['label']):
        points = data[image_id]['points'][idx]
        x_coords, y_coords = zip(*points)
        x_coords = np.array(x_coords, dtype=float)
        y_coords = np.array(y_coords, dtype=float)
        ax.plot(x_coords, y_coords, color=COLOR[label], linewidth=2)
    # Show the plot
    plt.show()

In [ ]:

def inverse_interpolate(data,image_id,im):
        size=(128,255)
        data_new = copy.deepcopy(data)
        inter_coeff = (size[1]/im.shape[2], size[0]/im.shape[1])
        for i, st in enumerate(data[str(image_id)]['stitches']):
            for j,pt in enumerate(st):
                pt = np.array(pt, dtype=float)
                pt *= inter_coeff
                # # hackfix to  ValueError: Expected x for keypoint ... to be in the range [0.0, 255], got 255.0.
                # for c in range(len(pt)):
                #     if pt[c] >= 255.0:
                #         pt[c] = 255.0 - 0.01
                data_new[str(image_id)]['stitches'][i][j] = list(pt)
        for i, pt in enumerate(data[str(image_id)]['incision']):
            pt = np.array(pt, dtype=float)
            pt *= inter_coeff
            # hackfix to  ValueError: Expected x for keypoint ... to be in the range [0.0, 255], got 255.0.
            for c in range(len(pt)):
                if pt[c] >= 255.0: pt[c] = 255.0 - 0.01
            data_new[str(image_id)]['incision'][i] = list(pt)

In [ ]:
if __name__ == "__main__":
    args=sys.argv
    output_file=args[1]
    visual_mode=True
    if args[2]=="-v":
      visual_mode=True
      input_imgs=args[3:]
    else:
      input_imgs=args[2:]
    for img in input_imgs:
      image=cv2.imread(img)
      interpolated_image=F.interpolate(image, size=(3,128,255)) # size=(3,128,255)

      #TODO:Metoda, která má jako vstup název obrázku, vrací json s incision a stiches

      f = open('data_processed.json')
      input_json=json.load(f)
      img_no="53"
      incision=[input_json[img_no]["incision"][0],input_json[img_no]["incision"][-1]]
      stitches=input_json[img_no]["stitches"]


      #nalezení pruseciku
      stitch_intersections=[]
      crossing_angles=[]
      for stitch in stitches:
        stitch_intersections.append(intersectLines( [float(incision[0][0]),float(incision[0][1])], [float(incision[1][0]),float(incision[1][1])],
                              [float(stitch[0][0]),float(stitch[0][1])], [float(stitch[1][0]),float(stitch[1][1])] ))
        

        #urceni uhlu ktery spolu svírají jizva a stehy
        lineA = ([float(incision[0][0]),float(incision[0][1])], [float(incision[1][0]),float(incision[1][1])])
        lineB = ([float(stitch[0][0]),float(stitch[0][1])], [float(stitch[1][0]),float(stitch[1][1])])

        slope1 = slope(lineA[0][0], lineA[0][1], lineA[1][0], lineA[1][1])
        slope2 = slope(lineB[0][0], lineB[0][1], lineB[1][0], lineB[1][1])

        crossing_angles.append(abs(angle(slope1, slope2)))#?????
      crossing_positions=[]  
      for x in stitch_intersections:
        crossing_positions.append(math.sqrt((float(incision[0][0])-x[0])**2+(float(incision[0][1])-x[1])**2))
      print(crossing_positions)


      

        

      data = [
          { "filename": img_no,
            "incision_polyline": [[ float(incision[0][0]), float(incision[0][1])],[float(incision[1][0]),float(incision[1][1])]],
            "crossing_positions": crossing_positions,
            "crossing_angles": crossing_angles,
          }, 
        ]


      with open('data.json', 'w', encoding='utf-8') as f:
          json.dump(data, f, ensure_ascii=False, indent=4)
    
      if visual_mode:
        visualize(input_json,img_no)







